### ONCE to poses (runtime: ~ 3min)

In [1]:
import json
import math
import os
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict

#### Set paths
<div class="alert alert-block alert-warning">
<b>ToDo:</b> 
<ol>
    <li> Set "datasets_root" to the root of your <b>dataset directory</b>
    <li> Set "vanishing_data_root" to your <b>working directory</b>
    <li> Select if <b>oxts</b> should be saved (runtime: ~ 30h)
</ol>
</div>

In [2]:
datasets_root = '/disk/ml/datasets'
vanishing_data_root = f'/disk/vanishing_data/ju878'

save_oxts = False

In [3]:
once_root = os.path.join(datasets_root, 'ONCE/data_root/data')

prepared_poses = os.path.join(vanishing_data_root, 'prepared_poses')
oxts_folder = os.path.join(prepared_poses, 'once/oxts')
poses_folder = os.path.join(prepared_poses, 'once/poses')

In [4]:
EARTH_RADIUS_METERS = 6.378137e6
REFERENCE_COORDINATES = {
    "zero": [0.000000000000, 0.000000000000]
}

In [5]:
if not os.path.exists(oxts_folder) and save_oxts:
        os.makedirs(oxts_folder)
if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)

#### Functions to create oxts and poses

Get poses

In [6]:
def get_poses(frames) -> List[dict]:
    pose_list = []
    for index, frame in enumerate(frames):
        pose_list.append(frame['pose'])
    
    return pose_list

Get coordinates (not real coordinates, relative to (0, 0))

In [7]:
def get_coordinate(ref_lat: float, ref_lon: float, bearing: float, dist: float) -> Tuple[float, float]:
    lat, lon = math.radians(ref_lat), math.radians(ref_lon)
    angular_distance = dist / EARTH_RADIUS_METERS
    
    target_lat = math.asin(
        math.sin(lat) * math.cos(angular_distance) + 
        math.cos(lat) * math.sin(angular_distance) * math.cos(bearing)
    )
    target_lon = lon + math.atan2(
        math.sin(bearing) * math.sin(angular_distance) * math.cos(lat),
        math.cos(angular_distance) - math.sin(lat) * math.sin(target_lat)
    )
    return math.degrees(target_lat), math.degrees(target_lon)

Get shortened oxts (only values needed for poses)

In [8]:
def derive_latlon(location: str, poses: List[Dict[str, float]], scene_name, frames) -> List[Dict[str, float]]:
    assert location in REFERENCE_COORDINATES.keys(), \
        f'Error: The given location: {location}, has no available reference.'
        
    oxts_scene_folder = os.path.join(oxts_folder, scene_name)
    oxts = []
    
    if save_oxts:
        if not os.path.exists(oxts_scene_folder):
            os.makedirs(oxts_scene_folder)
    
    reference_lat, reference_lon = REFERENCE_COORDINATES[location]
    for i, p in enumerate(poses):
        x, y, z = p[4:]
        if y == 0:
            bearing = 0.0
        else:
            bearing = math.atan(x / y)
        distance = math.sqrt(x**2 + y**2)
        lat, lon = get_coordinate(reference_lat, reference_lon, bearing, distance)
        
        r1, r2, r3, w = p[:4]
        roll = math.atan2(2 * (w * r1 + r2 * r3), 1 - 2 * (math.pow(r1, 2) + math.pow(r2, 2)))
        pitch = math.asin(2 * (w * r2 - r3 * r1))
        yaw = math.atan2(2 * (w * r3 + r1 * r2), 1 - 2 * (math.pow(r2, 2) + math.pow(r3, 2)))
        
        oxts.append([lat, lon, z, roll, pitch, yaw])
        
        if save_oxts:
            with open(os.path.join(oxts_scene_folder, frames[i]['frame_id'] + '.txt'), 'w') as f:
                f.write(f'{lat} {lon} {z} {roll} {pitch} {yaw}')
    return oxts

Helper functions

In [9]:
def lat_lon_to_mercator(lat, lon, scale):
    ER = 6378137
    mx = scale * lon * math.pi * ER / 180
    my = scale * ER * math.log(math.tan((90 + lat) * math.pi / 360))
    return mx, my


def lat_to_scale(lat):
    return math.cos(lat * math.pi / 180.0)

Translate oxts to poses

In [10]:
def gps_imu_to_pose(gps_imu, scale):
    t = np.zeros(3)
    t[0], t[1] = lat_lon_to_mercator(gps_imu[0], gps_imu[1], scale)
    t[2] = gps_imu[2]  # altitude
    rx = gps_imu[3]  # roll
    ry = gps_imu[4]  # pitch
    rz = gps_imu[5]  # heading
    Rx = np.array([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]])
    Ry = np.array([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]])
    Rz = np.array([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def oxts_to_pose(oxts_info):
    # Compute scale from first lat value
    scale = lat_to_scale(oxts_info[0][0])
    Tr_0_inv = None
    poses = np.zeros((len(oxts_info), 12))
    for i, oxts in enumerate(oxts_info):
        T = gps_imu_to_pose(oxts, scale)
        # Normalize translation and rotation (start at 0/0/0)
        if Tr_0_inv is None:
            Tr_0_inv = np.linalg.inv(T)

        pose = Tr_0_inv.dot(T)
        poses[i] = pose[:3, :].reshape(12)
    return poses

#### Translate ONCE_poses to KITTI_poses

In [ ]:
for scene in tqdm(range(0, 581)):
    scene_name = '{:0>6}'.format(scene)
    json_once_file = os.path.join(once_root, scene_name, scene_name + '.json')
    with open(json_once_file, 'r') as f:
        once_json = json.load(f)
     
    frames = once_json['frames']
    
    once_poses = get_poses(frames)  # For each pose, we will extract the corresponding coordinate.
    # Compute and store coordinates.
    oxts = derive_latlon('zero', once_poses, scene_name, frames)
    
    poses = oxts_to_pose(oxts)
    np.savetxt(os.path.join(poses_folder, scene_name + '.txt'), poses)